### Выполнила Зыкова Вероника, БКЛ-192

Необходимые импорты

In [1]:
import json
import pymorphy2
import pymystem3
from nltk.tokenize import word_tokenize
morph = pymorphy2.MorphAnalyzer()

from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

Читаем размеченные данные. Была выбрана разметка UD с единственным отличием - убран тег PROPN (так как он не определеяется частью остальных парсеров и не так сильно отличается от обычного NOUN, как SCONJ от CCONJ, например). Эта разметка была выбрана как наиболее лаконичная и при этом удобнаая как для целей морфологического анализа, так и для синтаксиса.

In [2]:
with open('corpus.json', 'r', encoding='utf-8') as f:
    corpus = json.load(f)
corpus = [tuple(el) for el in corpus if el[1] != 'PUNCT']

Читаем сырой текст. Данные были взяты из трех источников: 
1. Отчет лаборатории истории диаспор исторического факультета МГУ (в тексте много аббревиатур и имен, что является сложным для парсеров)
2. Описание лемматизация и стемминга с сайта, посвященного программированию (термины и заимствования в качестве сложностей)
3. Описание инструментов для каллиграфии (редкие слова и заимствовавния)


In [3]:
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

Разбор частей речи через pymorphy

In [4]:
def pymorphy_par(text):
    tokens = word_tokenize(text)
    res = []
    for token in tokens:
        token_pr = morph.parse(token.lower())[0] 
        pos = str(token_pr.tag.POS)
        res.append((token, pos))
    return res

Приводим разметку к нужной (функция универсальна для всех трех парсеров)

In [5]:
def rename(posses, conn):
    new = []
    for el in posses:
        if el[1] in conn:
            new.append((el[0], conn[el[1]]))
        else:
            new.append(el)
    return new

Сопостравляем токенизацию с той, которая есть в собранных данных (функция универсальна для всех трех парсеров)

In [6]:
def connect(posses, corpus):
    res = []
    i = 0
    cur = ''
    for el in corpus:
        stop = False
        while not stop:
            if i >= len(posses):
                stop = True
            elif posses[i][0] == el[0]:
                stop = True
                res.append(posses[i])
                i += 1
            else:
                if el[0].startswith(posses[i][0]):
                    stop = True
                    cur += posses[i][0]
                    i += 1
                    while cur:
                        if el[0].endswith(posses[i][0]):
                            cur += posses[i][0]
                            res.append((cur, posses[i][1]))
                            cur = ''
                            i += 1
                        else:
                            cur += posses[i][0]
                            i += 1
                else:
                    i += 1
    return res

Собираем весь разбор pymorphy

In [7]:
pm = pymorphy_par(text)
conn = {'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADJ', 'INFN': 'VERB', 'PRTF': 'ADJ', 'PRTS': 'VERB', 'GRND': 'VERB', 
            'NUMR': 'NUM', 'ADVB': 'ADV', 'NPRO':'PRON', 'PRED': 'ADV', 'PREP': 'ADP', 'PRCL': 'PART'}
pm = rename(pm, conn)
pm = connect(pm, corpus)

Части речи mystem

In [8]:
def mystem_par(text):
    m = pymystem3.Mystem()
    an = m.analyze(text)
    res = []
    for el in an:
        s = el['text']
        if 'analysis' in el:
            pos = el['analysis'][0]['gr']
            pos = pos.split('=')[0]
            pos = pos.split(',')[0]
        else:
            pos = None
        res.append((s, pos))
    return res

Собираем весь mystem вместе

In [9]:
ms = mystem_par(text)
conn = {'A': 'ADJ', 'ADVPRO': 'ADV', 'ANUM': 'NUM', 'APRO': 'DET', 'PR': 'ADP', 'V': 'VERB', 'SPRO':'PRON', 'S': 'NOUN'}
ms = rename(ms, conn)
ms = connect(ms, corpus)

Части речи Наташи

In [10]:
def nata_par(text):
    emb = NewsEmbedding()
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    
    morph_tagger = NewsMorphTagger(emb)
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    res = []
    for el in doc.tokens:
        if el.pos != 'PUNCT':
            res.append((el.text, el.pos))
    
    return res

Собираем Наташу

In [11]:
nt = nata_par(text)
conn = {'PROPN': 'NOUN'}
nt = rename(nt, conn)
nt = connect(nt, corpus)

Функция оценки результатов работы

In [12]:
def get_res(posses, curpus, mist=False):
    res = {mod: 0 for mod in posses.keys()}
    for i, el in enumerate(corpus):
        for mod in posses.keys():
            if el[1] == posses[mod][i][1]:
                res[mod] += 1
            elif ((el[1] == 'SCONJ') or (el[1] == 'CCONJ')) and (posses[mod][i][1] == 'CONJ'):
                res[mod] += 1
            elif (el[1] == 'AUX') and (posses[mod][i][1] == 'VERB'):
                res[mod] += 1
            else:
                if mist:
                    print(f'Mistake: {mod} -- {el} -- {posses[mod][i]}')
    for el in res:
        res[el] = res[el] / len(corpus)
    return res

Собираем всю выдачу в один словарь

In [13]:
posses = {'natasha': nt, 'mystem': ms, 'pymorphy': pm}

Результаты разборов

In [14]:
get_res(posses, corpus)

{'natasha': 0.8984615384615384,
 'mystem': 0.9323076923076923,
 'pymorphy': 0.8861538461538462}

N-граммы для поиска

In [15]:
grams = ['не VERB', 'VERB NOUN', 'NOUN VERB NOUN']

Функция поиска н-грамм

In [16]:
def find_grams(text, grams):
    res = {gr: [] for gr in grams}
    gram = {}
    for el in grams:
        pat = el.split()
        pat_nums = [int(i.isupper()) for i in pat]
        pats = [(pat[i], pat_nums[i]) for i in range(len(pat))]
        gram[el] = [pats, 0]
    
    cur = {gr: [] for gr in grams}
    for el in text:
        for gr in gram:
            x = gram[gr][0][gram[gr][1]][1]
            w = gram[gr][0][gram[gr][1]][0]
            if el[x].lower() == w.lower():
                cur[gr].append(el[0])
                gram[gr][1] += 1
            else:
                if len(cur[gr]) > 0:
                    cur[gr] = []
                    gram[gr][1] = 0
            if len(cur[gr]) == len(gram[gr][0]):
                res[gr].append(' '.join(cur[gr]))
                cur[gr] = []
                gram[gr][1] = 0
                
    return res

Результаты поиска

In [17]:
find_grams(corpus, grams)

{'не VERB': ['не понимает', 'не иметь'],
 'VERB NOUN': ['курирует лаборатория',
  'состоялась встреча',
  'отличаются Стемминг',
  'использует словарь',
  'привести слово',
  'понимает разницу',
  'иметь значения',
  'соблюдать наклон',
  'разработать руку'],
 'NOUN VERB NOUN': ['итоге привести слово']}

Остальное см. в тетрадке Hw1_nlp_2 в этой же папке.